# Part 1: EDA

**1. Overview of dataset [15%]**
a. Summarize the background of the dataset.
b. State the size of the dataset.
c. For each variable, describe what it represents and its data type (numerical or categorical).

**2. Data pre-processing [35%]**
a. For each variable, determine the percentage of missing data. For any column with missing data, describe how you resolve the issue. Clearly state any assumption you made.
b. For each variable, identify outliers (if any) and describe how you resolve the issue. Clearly state any assumption you made.
c. For categorical variables, perform the necessary encoding.

**3. Exploratory analysis and visualization [50%]**
a. For each variable, provide relevant summary statistics.
b. For each variable, provide an appropriate visualisation depicting the distribution of its values, and summarize any key observation(s) you made.
c. Perform bi-variate analyses on the variables. You do not need to analyse every pair; only focus on the pairs you believe are worth investigating and explain your choices. For each pair, describe the relationship between the two variables. Use appropriate statistical methods and/or visualization.

If applicable, corresponding codes that are reproducible (i.e., they produce output consistent with your answers), and well documented (in the form of comments and markdown cells).

In [ ]:
# Imported libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Reading of Dataset

In [ ]:
df = pd.read_csv("employee.csv")
df

## 1. Overview of dataset

### a. Summarize the background of the dataset.
The dataset contains HR data related to a sales team. Specifically the dataset captures employees’ information at the beginning of each month, including if the employee quit in that given month, total sales acquired so far in his/her career and his/her latest quarterly rating. The dataset contains 2381 rows and 13 columns.

| Column Name | Description | Type | Details |
| ----------- | ----------- | ---- | ------- |
| Date | Record capture date | Categorical | Date in format DD/MM/YYYY |
| Emp_ID | Employee ID | Categorical |
| Age | Age | Numerical |
| Gender | Gender | Categorical | "Male", "Female" |
| City | City where employee works | Categorical | "C1", "C2", ..., "C29" |
| Education | Highest education level of employee | Categorical | "College", "Bachelor", "Master" |
| Salary | Last drawn salary | Numerical |
| Join_Date | Date of joining the sales team | Categorical | Date in format DD/MM/YYYY |
| Last_Work_Date | Last working date with the sales team | Categorical | Date in format DD/MM/YYYY. If employee does not quit in the given month where record is captured, the field would be empty (NaN). |
| Join_Designation | Designation level when first joined | Categorical | 1, 2, 3, 4, 5. |
| Designation | Designation level at the time record is captured | Categorical | 1, 2, 3, 4, 5 |
| Total_Sales_Acquired | Total sales generated by employee since joining the team | Numerical |
| Quarterly_Rating | Latest quarterly performance rating | Categorical | 1, 2, 3, 4 |



In [ ]:
# get rows and columns (size) of dataframe
n_rows, n_cols = df.shape

print(f"Number of rows: {n_rows}")
print(f"Number of columns: {n_cols}")

In [ ]:
# get datatype of attributes/features
df.dtypes

In [ ]:
# analysing unique values of specific columns
genders = df["Gender"].unique()
cities = df["City"].unique()
education_levels = df["Education"].unique()
join_designation_levels = df["Join_Designation"].unique()
designation_levels = df["Designation"].unique()
quarterly_ratings = df["Quarterly_Rating"].unique()

print(f"Possible values of 'Gender' attribute: {genders}")
print(f"Possible values of 'City' attribute: {cities}")
print(f"Possible values of 'Education' attribute: {education_levels}")
print(f"Possible values of 'Join_Designation' attribute: {join_designation_levels}")
print(f"Possible values of 'Designation' attribute: {designation_levels}")
print(f"Possible values of 'Quarterly_Rating' attribute: {quarterly_ratings}")

## 2. Data pre-processing

In [ ]:
# number of available data
df.count()

In [ ]:
# percentage of data missing per attribute
print("Percentage of data missing")

col = df.columns
i = 0
missing_data_col = []

for valid_data in df.count():
    print(col[i], end=" = ")
    per_missing = (1 - (valid_data/n_rows))*100
    print(f"{per_missing : .2f}%")

    if per_missing > 0:
        missing_data_col.append(col[i])
        
    i += 1
print()

# print all column names of columns with missing data
print("Columns with missing data: " + str(missing_data_col))

### Missing Data
| Column Name | Percentage Missing | Resolution Technique | Assumptions |
| ----------- | ------------------ | -------------------- | ----------- |
| Last_Work_Date | 32.13% | Impute NaN values as the date in the "Date" attribute and Impute a new attribute "Current_Staff" with those that have NaN values initialised as "Yes" and those that have Last_Work_Date as "No". | Fields with NaN values suggest that the staff is still with the company. |
| Join_Date | 4.96% | Impute NaN values as the average tenure of their salary group from their Last_Work_Date | There is a correlation between salary and the tenure length of a worker in the company. |
| Join_Designation | 4.41% | Impute the NaN values as 1. | All workers begin at the lowest Designation level when they joined the company. |

In [ ]:
# Resolving Last_Work_Date
print(f"Last_Work_Date has been resolved: {df["Last_Work_Date"].count() == n_rows}")
df["Current_Staff"] = "No"
df.loc[df["Last_Work_Date"].isna(), "Current_Staff"] = "Yes"
df.loc[df["Last_Work_Date"].isna(), "Last_Work_Date"] = df["Date"]
print(f"Last_Work_Date has been resolved: {df["Last_Work_Date"].count() == n_rows}")

In [ ]:
from datetime import datetime
print(f"Join_Date has been resolved: {df["Join_Date"].count() == n_rows}")

# Convert date columns to datetime
df['Join_Date'] = pd.to_datetime(df['Join_Date'], format='%d/%m/%Y', errors='coerce')
df['Last_Work_Date'] = pd.to_datetime(df['Last_Work_Date'], format='%d/%m/%Y', errors='coerce')

# Filter employees with missing Join_Date
missing_join_df = df[df['Join_Date'].isna()].copy()

# Filter employees with both Join_Date and Last_Work_Date
valid_join_df = df[df['Join_Date'].notna() & df['Last_Work_Date'].notna()].copy()

# Calculate tenure in days for employees with valid join and last work dates
valid_join_df.loc[:, 'Tenure'] = (valid_join_df['Last_Work_Date'] - valid_join_df['Join_Date']).dt.days

# Group by salary ranges (e.g., every 10,000)
valid_join_df.loc[:, 'Salary_Range'] = pd.cut(valid_join_df['Salary'], bins=range(0, 150000, 10000))

# Calculate average tenure for each salary range
avg_tenure_by_salary = valid_join_df.groupby('Salary_Range', observed=True)['Tenure'].mean()

# Function to estimate join date based on salary range
def estimate_join_date(row):
    salary_range = pd.cut([row['Salary']], bins=range(0, 150000, 10000))[0]
    if salary_range in avg_tenure_by_salary:
        avg_tenure = avg_tenure_by_salary[salary_range]
        if pd.notna(row['Last_Work_Date']):
            estimated_join_date = row['Last_Work_Date'] - pd.Timedelta(days=avg_tenure)
            return estimated_join_date
    return row['Last_Work_Date'] - pd.DateOffset(years=1)  # Default fallback

# Update only the rows with missing join dates
df.loc[missing_join_df.index, 'Join_Date'] = missing_join_df.apply(estimate_join_date, axis=1)

# Convert the estimated join date back to the original format
df['Join_Date'] = df['Join_Date'].dt.strftime('%d/%m/%Y')
df['Last_Work_Date'] = df['Last_Work_Date'].dt.strftime('%d/%m/%Y')

# Print the updated DataFrame
# print(df[['Emp_ID', 'Salary', 'Date', 'Join_Date']])
# print(df['Join_Date'].count() / df['Emp_ID'].count() * 100)

# Verify all Join_Date values are filled
print(f"Join_Date has been resolved: {df['Join_Date'].count() == n_rows}")

In [ ]:
# Resolving Join_Designation
print(f"Join_Designation has been resolved: {df["Join_Designation"].count() == n_rows}")
df["Join_Designation"] = df["Join_Designation"].replace(np.nan, 1)

print(f"Join_Designation has been resolved: {df["Join_Designation"].count() == n_rows}")

In [ ]:
# check if all NaN values have been handled

df.count()

### Invalid Data Identification

We will now be checking if there are any outliers in the dataset provided. 

We will do so by checking the min and max value of each variables and see if there are any values that is outside of the norm of each variable.

In [ ]:
# checking the min of each variable
print("Min value of each variable\n")
col = df.columns
for i in range(len(df.columns)):
    print(col[i], end=" = ")
    min_value = df[col[i]].min()
    print(str(min_value))


As seen from the results, the minimum value of the Total_Sales_Acquired attribute is negative. This is an outlier as we assumed that employees can only sales or no sales at all. There should not be employees that make negative sales.

Hence, we will be removing all the employees that are making negative sales.

In [ ]:
# checking the number of employees that make negative sales
negative_sales = df.loc[df["Total_Sales_Acquired"] < 0, "Total_Sales_Acquired"].count()
print("Employees with negative sales: " + str(negative_sales))
print("Percentage of employees with negative sales out of all employees: " + str((negative_sales / df['Total_Sales_Acquired'].count())*100) + "%")

# removing the aforementioned employees from the data
df = df.drop(df[df['Total_Sales_Acquired'] < 0].index)

# check again
print("Updated employees with negative sales: " + str(df.loc[df["Total_Sales_Acquired"] < 0, "Total_Sales_Acquired"].count()))
print("Min. Total Sales Acquired: " + str(df["Total_Sales_Acquired"].min()))

We will now be checking the max value of each variables and see if there are any values that is outside of the norm of each variable.

In [ ]:
# checking the max of each variable
print("Max value of each variable\n")
col = df.columns
for i in range(len(df.columns)):
    print(col[i], end=" = ")
    min_value = df[col[i]].max()
    print(str(min_value))

In [ ]:
# number of available data
print(df.count())
df

### How we resolved the invalid data

| Column Name | Problem | Percentage Invalid | Resolution Technique | Assumptions |
| ----------- | ------- | ------------------ | -------------------- | ----------- |
| Total_Sales_Acquired| Some of the records have Total_Sales_Acquired that are negative | 0.41999160016799664% | Since there are only 10 or 0.420% of the total records that show invalid data, we have decided to remove those records. | We assume that negative Total_Sales_Acquired is not possible, hence suggesting that the data is invalid. |

### Outlier Identification

We must now analyze the dataset provided, and identify the outliers from selected variables.

The selection process lies solely on intuition. For example, we chose age for there will, of course, be a possibility that an employee would be outside of the hiring norms.

All of these side, we chose three variables to analyze:
1. Age
2. Salary
3. Sales

For each variable, we identify the following:
1. 25th Quartile
2. 75th Quartile
3. Interquartile range (IQR)

And use the obtained information to identify the upper and lower boundaries. With these on hand, we can then identify the outliers by checking if the data goes beyond the boundaries.

In [ ]:
## Age
# quantiles
AGEQ1 = df["Age"].quantile(0.25)
AGEQ3 = df["Age"].quantile(0.75)
AGEIQR = AGEQ3-AGEQ1

# boundaries
AGEUpper = AGEQ3+(1.5*AGEIQR)
AGELower = AGEQ1-(1.5*AGEIQR)
AGEOutlier = df[(df["Age"]<AGELower)|(df["Age"]>AGEUpper)]
print("AGE 25th Percentile:", AGEQ1)
print("Youngest employee:", df["Age"].min())
print("AGE 75th Percentile:", AGEQ3)
print("Oldest employee:", df["Age"].max())
print("AGE IQR:", AGEIQR)
print("Total amount of AGE Outliers:", len(AGEOutlier))

# this one prints everything idk how to pinpoint the employee ids only
# print("AGE Outliers", AGEOutlier)

In [ ]:
## Salary
# quantiles
SALQ1 = df["Salary"].quantile(0.25)
SALQ3 = df["Salary"].quantile(0.75)
SALIQR = SALQ3-SALQ1

# boundaries
SALUpper = SALQ3+(1.5*SALIQR)
SALLower = SALQ1-(1.5*SALIQR)
SALOutlier = df[(df["Salary"]<SALLower)|(df["Salary"]>SALUpper)]
print("SALARY 25th Percentile: $", SALQ1)
print("Lowest salary: $", df["Salary"].min())
print("SALARY 75th Percentile: $", SALQ3)
print("Highest salary: $", df["Salary"].max())
print("Salary IQR: $", SALIQR)
print("Total amount of Salary Outliers:", len(SALOutlier))

# this one prints everything idk how to pinpoint the employee ids only
# print("Salary Outliers", SALOutlier)

In [ ]:
## Salary
# quantiles

# need to filter again for zeroes if thats a concern
# i understand that negatives got removed
# this one includes zeroes

SALESQ1 = df["Total_Sales_Acquired"].quantile(0.25)
SALESQ3 = df["Total_Sales_Acquired"].quantile(0.75)
SALESIQR = SALESQ3-SALESQ1

# boundaries
SALESUpper = SALESQ3+(1.5*SALESIQR)
SALESLower = SALESQ1-(1.5*SALESIQR)
SALESOutlier = df[(df["Total_Sales_Acquired"]<SALESLower)|(df["Total_Sales_Acquired"]>SALESUpper)]
print("SALES 25th Percentile: $", SALESQ1)
print("Lowest sales: $", df["Total_Sales_Acquired"].min())
print("SALES 75th Percentile: $", SALESQ3)
print("Highest sales: $", df["Total_Sales_Acquired"].max())
print("Sales IQR: $", SALESIQR)
print("Total amount of Sales Outliers:", len(SALESOutlier))

# this one prints everything idk how to pinpoint the employee ids only
# print("Salary Outliers", SALOutlier)

## 3. EDA and Visualisation

1. Univariate
- histogram for Total_Sales_Acquired
- histogram for Total_Sales_Acquired without employees with 0 sales
- donut chart for Designation_Level distribution
- pie chart for Quarterly_Rating distribution
- histogram for Salary distribution
- histogram for Age distribution
- pie chart for Gender distribution
- pie chart for Education level distribution 
- bar chart for City distribution 


2. Bivariate
- grouped boxplot for Salary against Education level
- grouped boxplot for Salary against designation
- grouped boxplot for Salary against Gender
- scatter plot for Salary against Age
- grouped boxplot for Total_Sales_Acquired and Gender
- scatter plot for Total_Sales_Acquired against Age
- scatter plot for Total_Sales_Acquired against Salary
- histogram for Total_Sales_Acquired by Designation_Level 
- histogram for Total_Sales_Acquired by Quarterly_Rating 

## Univariate Analysis

### Total_Sales_Acquired
With a large standard deviation, it goes to show that the spread of the Total_Sales_Acquired across all employees in the company is rather large. 

The histogram plot also suggests that employees either produce very little to no sales or a lot of sales. Perhaps this suggests that the company has a significant proportion of employees that are not involved in sales. For more meaningful analysis, it could perhaps be more useful to categorise the type of employees.

Alternatively, this might also indicates that some employees may be facing challenges or are underperforming. However, there is also a group of employees that achieved high sales reaching 10-18 which gives a clear difference between low and high performers.


In [ ]:
# mean
print(f"Mean total sales acquired: {df["Total_Sales_Acquired"].mean() : .2f}")

# standard deviation
print(f"s.d. total sales acquired: {df["Total_Sales_Acquired"].std() : .2f}")

# min
print(f"Min total sales acquired: {df["Total_Sales_Acquired"].min() : .2f}")

# median 
print(f"Median total sales acquired: {df["Total_Sales_Acquired"].median() : .2f}")

# max
print(f"Max total sales acquired: {df["Total_Sales_Acquired"].max() : .2f}")

In [ ]:
# Histogram of Total Sales Acquired
plt.figure(figsize=(8,5))
plt.title("Total Sales Acquired", fontsize=15)
plt.xlabel("Sales", fontsize=12)
plt.ylabel("Num of Employees", fontsize=12)
log_TotalSales = np.log1p(df.Total_Sales_Acquired)
log_TotalSales.hist(bins=10) 
plt.show()

### Total_Sales_Acquired without Employees with 0 sales

With the revelation that there are many employees that have 0 sales, we decided to do a total sales acquired analysis only on employees that has sales so that the insights gathered are more meaningful.

Excluding the 719 employees with 0 sales, the mean total sales is actually about 2 million more than previously analysed. This shows that the employees making sales are doing a much better job than previously analysed. Furthermore, with reference to the histogram, the total sales acquired does follow the a normal distribution. 

In [ ]:
# number of employees with 0 sales
print("Number of employees with 0 sales: " + str(df.loc[df["Total_Sales_Acquired"] == 0, "Total_Sales_Acquired"].count()))

# mean
print(f"Mean total sales acquired: {df.loc[df["Total_Sales_Acquired"] > 0, "Total_Sales_Acquired"].mean() : .2f}")

# standard deviation
print(f"s.d. total sales acquired: {df.loc[df["Total_Sales_Acquired"] > 0, "Total_Sales_Acquired"].std() : .2f}")

# min
print(f"Min total sales acquired: {df.loc[df["Total_Sales_Acquired"] > 0, "Total_Sales_Acquired"].min() : .2f}")

# median 
print(f"Median total sales acquired: {df.loc[df["Total_Sales_Acquired"] > 0, "Total_Sales_Acquired"].median() : .2f}")

# max is the same as above

In [ ]:
# Histogram of Total Sales Acquired
plt.figure(figsize=(8,5))
plt.title("Total Sales Acquired without Employees with 0 sales", fontsize=15)
plt.xlabel("Sales", fontsize=12)
plt.ylabel("Num of Employees", fontsize=12)
log_TotalSales = np.log1p(df.loc[df["Total_Sales_Acquired"] > 0, "Total_Sales_Acquired"])
log_TotalSales.hist(bins=10)  
plt.show()

### Designation_Level
It suggests that the Designation level 5 could correspond to the Directors of the company and with each Designation level in descending order suggesting a more junior level staff of the company. This is representative of a typical corporate organisational structure.

In [ ]:
# mean
print(f"Mean designation level: {df["Designation"].mean() : .2f}")

# standard deviation
print(f"s.d. designation level: {df["Designation"].std() : .2f}")

# min
print("Min designation level: " + str(df["Designation"].min()))

# median 
print("Median designation level: " + str(df["Designation"].median()))

# max
print("Max designation level: " + str(df["Designation"].max()))



In [ ]:
# Doughnut Chart showing the Percentage of Workers in Each Designation Level
plt.figure(figsize=(6, 6))
plt.pie(df["Designation"].value_counts().sort_index(), labels=df["Designation"].unique(), autopct='%1.1f%%', wedgeprops={'edgecolor': 'white'}, pctdistance=.8)

# Create a circle at the center to make it a doughnut
centre_circle = plt.Circle((0, 0), 0.40, fc='white')
plt.gca().add_artist(centre_circle)

# Title & Show Plot
plt.title("Percentage of Workers in Each Designation Level")
plt.show()

### Quarterly_Rating
The pie chart shows a breakdown of the proportion of employees in for each Quarterly_Rating band. 

Intuitively, this suggests that band 4 which makes up the smallest proportion of employees at 4.5% are the top performers for the quarter. A vast majority of employees are awarded band 1 at 73.2% of the all employees in the company.

This noticeable performance difference where a majority of employees received the lowest rating, might indicate widespread underperformance. The small number of employees with high ratings can also further suggest challenges in performance and lack of staff training.


In [ ]:
# mean
print(f"Mean quarterly rating: {df["Quarterly_Rating"].mean() : .2f}")

# standard deviation
print(f"s.d. quarterly rating: {df["Quarterly_Rating"].std() : .2f}")

# min
print("Min quarterly rating: " + str(df["Quarterly_Rating"].min()))

# median 
print("Median quarterly rating: " + str(df["Quarterly_Rating"].median()))

# max
print("Max quarterly rating: " + str(df["Quarterly_Rating"].max()))

In [ ]:
# Pie Chart showing the Percentage of Workers with Each Quarterly Rating
plt.figure(figsize=(6, 6))
plt.pie(df["Quarterly_Rating"].value_counts().sort_index(), labels=sorted(df["Quarterly_Rating"].unique()), autopct='%1.1f%%', wedgeprops={'edgecolor': 'white'}, pctdistance=.6)

# Title & Show Plot
plt.title("Percentage of Workers with Each Quarterly Rating")
plt.show()

### Salary of Employees
The mean salary of employees in the company is $59336.13. Based on the histogram, the distributions of salary of employees seem to follow a normal distribution.

In [ ]:
print(f"Mean employee salary: {df['Salary'].mean() : .2f}")

print(f"s.d. employee salary: {df['Salary'].std() : .2f}")

# min
print("Min employee salary: " + str(df["Salary"].min()))

# median 
print("Median employee salary: " + str(df["Salary"].median()))

# max
print("Max employee salary: " + str(df["Salary"].max()))

In [ ]:
plt.figure(figsize=(8,5))
plt.title("Salary of Employees", fontsize=15)
plt.xlabel("Salary", fontsize=12)
plt.ylabel("Num of Employee", fontsize=12)
df["Salary"].hist(bins=20)  
plt.show()

### Age of Employees
Based on the histogram, it seems to imply that majority of the employees in the company are around 30 years of age. 

The mean age is 33.7 which suggests that the distribution of the age of employees tend more towards larger than 30 years of age than it does to less than 30 years of age.

In [ ]:
print(f"Mean age of employee: {df['Age'].mean() : .0f}")

print(f"s.d. age of employee: {df['Age'].std() : .0f}")

# min
print("Min age of employee: " + str(df["Age"].min()))

# median 
print("Median age of employee: " + str(df["Age"].median()))

# max
print("Max age of employee: " + str(df["Age"].max()))

In [ ]:
# Histogram for Age
plt.figure(figsize=(8,5))
plt.title("Age of Employees", fontsize=15)
plt.xlabel("Age", fontsize=12)
plt.ylabel("Num of Employee", fontsize=12)
# log_Age = np.log1p(df.Age)
df["Age"].hist(bins=10)  
plt.show()

### Gender Distribution
The pie chart shows the gender distribution among employees, revealing that 59.0% of the workforce is male, while 41.0% is female.

This illustrates a small gender gap with males taking the majority. The male-dominated environment may also be caused by industry trends, workplace policies or societal factors. Furthermore, it may also indicate a bias in hiring male employees because of the job requirements that are aligned more with male employees.


In [ ]:
# Pie Chart showing the Gender Distribution of the company
x = df['Gender'].value_counts()
y = x.index
plt.figure(figsize=(6, 6)) #size
plt.pie(x, labels=y, autopct='%1.1f%%') #string format
plt.title("Gender Distribution")
plt.show()

### Education Distribution
The pie chart illustrates the distribution of education levels of employees.

With reference to the pie chart, there is a fairly balanced proportion across the three education categories. 

The nearly equal proportions indicate that the company has employees from an evenly distributed educational background.

In [ ]:
x1 = df['Education'].value_counts()
y1 = x1.index
plt.figure(figsize=(6, 6)) #size
plt.pie(x1, labels=y1, autopct='%1.1f%%') #string format
plt.title("Education Distribution")
plt.show()

### City Distribution
With most of the employees stemming from city C20, it could suggest 2 outcomes: 

1. The company could be based in city C20, where they tend to be more comfortable in hiring employees close to their location

2. The company could be hiring employees from city C20 due to it being a metropolis.

In [ ]:
city_counts = df['City'].value_counts()

plt.figure(figsize=(16, 10))
plt.bar(city_counts.index, city_counts.values)

plt.title("City Distribution")
plt.xlabel("City")
plt.ylabel("Number of Employees")
plt.xticks(rotation=45) 

plt.show()


## Bivariate Analysis

### Education and Salary
With reference to the boxplot, it shows that the average salary between the various education level do not vary by a significant amount.

However, it can be seen that with a higher level of education, there is a strong correlation to greater salary as seen by a larger number of outliers receiving a higher salary for those employees that has a Master degree

In [ ]:
df['Salary'] = pd.to_numeric(df['Salary'], errors='coerce')

plt.figure(figsize=(10, 6))

education_levels = df['Education'].unique()

education_groups = [df[df['Education'] == level]['Salary'] for level in education_levels]

plt.boxplot(education_groups, tick_labels=education_levels)

plt.title('Bivariate Analysis: Education vs. Salary')
plt.xlabel('Education Level')
plt.ylabel('Salary')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

### Salary against Designation
With reference to the boxplot, there is a clear positive relationship between the amount of salary received and their designation as seen by the increasing mean salary of employees at a certain designation. 

However, as seen from the spread of each boxplot, it is interesting to note that having a higher designation level does not necessarily mean that one would have a higher salary

In [ ]:
df['Salary'] = pd.to_numeric(df['Salary'], errors='coerce')

plt.figure(figsize=(10, 6))

designation_levels = df['Designation'].unique()

designation_groups = [df[df['Designation'] == level]['Salary'] for level in designation_levels]

plt.boxplot(designation_groups, tick_labels=designation_levels)

plt.title('Bivariate Analysis: Salary vs Designation')
plt.xlabel('Designation Level')
plt.ylabel('Salary')

plt.tight_layout()
plt.show()

### Salary against Gender
The box plot provides a comparative analysis of the salary distribution by gender.

With reference to the boxplot, both genders exhibit similar median salaries, indicating that, on average, salaries are fairly balanced. 

However, there is a significant spread in salaries, with some outliers extending well beyond the upper quartile for both categories, suggesting that a portion of employees earns substantially more than the typical salary range, probably indicating that they are holding higher designation levels in the company (as seen from the boxplot of Salary vs Designation).

Furthermore, the presence of more extreme outliers among males may imply that a higher number of men occupy higher designation level or experience greater salary variation than women.

In [ ]:
df['Salary'] = pd.to_numeric(df['Salary'], errors='coerce')

plt.figure(figsize=(10, 6))

genders = df['Gender'].unique()

gender_groups = [df[df['Gender'] == gender]['Salary'] for gender in genders]

plt.boxplot(gender_groups, tick_labels=genders)

plt.title('Bivariate Analysis: Salary vs Gender')
plt.xlabel('Gender')
plt.ylabel('Salary')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

### Salary against Age
We wanted to confirm our hypothesis that an employee being older in terms on Age would incur a higher salary. We believed that Age would equate to more experience and thus command a higher salary. 

After creating the scatterplot, the regression line does show a slight linear relationship between Salary and Age, but the salary ranges seem evenly distributed across all age ranges.

In [ ]:
age = df['Age']
salary = df['Salary']
plt.figure(figsize=(12, 6))  
plt.scatter(age,salary)
plt.title("Plot of Salary vs Age")
plt.xlabel("Age")
plt.ylabel("Salary")
# drawing the regression line
line = 1000.01 * age + 25672.63
fig = plt.plot(age,line, lw=4, c='orange', label = 'Regression Line')
plt.show()

### Total_Sales_Acquired against Gender
After looking at the difference between salary of both genders, we were curious regarding the sales difference between the genders. 

With reference to boxplot, the median total sales appear to be similar for both genders, suggesting that, on average, there is no significant difference in sales performance between men and women. 

However, both distributions show a considerable number of outliers. This might be because of the large spread of the distribution of the total sales acquired as seen above. 


In [ ]:
df['Total_Sales_Acquired'] = pd.to_numeric(df['Total_Sales_Acquired'], errors='coerce')

plt.figure(figsize=(10, 6))

genders = df['Gender'].unique()

gender_groups = [df[df['Gender'] == gender]['Total_Sales_Acquired'] for gender in genders]

plt.boxplot(gender_groups, tick_labels=genders)

plt.title('Bivariate Analysis: Total Sales Acquired vs Gender')
plt.xlabel('Gender')
plt.ylabel('Total_Sales_Acquired')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

### Total_Sales_Acquired against Age
We wanted to confirm our hypothesis that an employee being older in terms on Age would incur more Total Sales Acquired. We believed that Age would equate to more experience, and thus be able to perform better in terms of sales. 

After creating the scatterplot, the regression line does show a slight linear relationship between Total Sales Acquired and Age, but it seems that the age where the most Total Sales would occur is around the age of 40. This could be because as employees grow older after 40, they may be less focused on their performance due to other factors such as family or health.

In [ ]:
age = df['Age']
sales = df['Total_Sales_Acquired']
plt.figure(figsize=(12, 6))  
plt.scatter(age,sales)
plt.title("Plot of Total Sales Acquired vs Age")
plt.xlabel("Age")
plt.ylabel("Total Sales Acquired")
# drawing the regression line
line2 = 400776.15 * age + -8905221.80
fig = plt.plot(age,line2, lw=4, c='orange', label = 'Regression Line')
plt.show()

### Total_Sales_Acquired against Salary
We wanted to investigate the relationship beteeen Total Sales Acquired and Salary, and believed that they are linearly related as a better sales performance would justify an increase in salary. 

After running the data through a scatterplot, we observed that there is a linear relationship between Total Sales Acquired and Salary, but the relationship may not be as strong as we thought, as there is a significant portion of outliers where Total Sales seems to not affect the Salary.

In [ ]:
salary = df["Salary"]
sales = df['Total_Sales_Acquired']
plt.figure(figsize=(12, 6))  
plt.scatter(salary,sales)
plt.title("Plot of Total Sales Acquired vs Salary")
plt.xlabel("Salary")
plt.ylabel("Total Sales Acquired")
# drawing the regression line
line3 = 122.09 * salary + -2657931.02
fig = plt.plot(salary,line3, lw=4, c='orange', label = 'Regression Line')
plt.show()

### Total_Sales_Acquired against Quarterly_Rating
We wanted to investigate the relationship between Total Sales Acquired and Quarterly Rating as we believed that having a higher rating will translate to have higher total sales.

The bar chart illustrates a positive correlation between the Quarterly Rating and the Mean Total Sales Achieved. Employees with higher quarterly ratings tend to achieve greater total sales on average. This trend indicates that as ratings improve, sales also increase, demonstrating that performance ratings are a reliable reflection of each employee's sales productivity.

In [ ]:
mean_sales = df.groupby("Quarterly_Rating")["Total_Sales_Acquired"].mean()

plt.figure(figsize=(8,5))
plt.bar(mean_sales.index, mean_sales.values)

plt.xlabel("Quarterly Rating", fontsize=12)
plt.ylabel("Mean Total Sales Acquired", fontsize=12)
plt.title("Mean Total Sales Acquired vs. Quarterly Rating", fontsize=14)
plt.xticks(mean_sales.index)  # Ensure x-axis labels match the rating values

plt.show()

### Total_Sales_Acquired against Designation_Level
We wanted to investigate the relationship between Total Sales Acquired and Designation as we believed that they are related as a higher the designation will reflect a greater total sales they made. 

The bar chart illustrates the relationship between total sales and employee designation levels. It shows a clear upward trend, indicating that as designation levels increase, total sales also rise. Employees in designation level 1 have the lowest total sales, while those in designation level 5 achieve the highest sales. This suggests that higher designations, which likely correlate with increased experience, responsibilities, and authority, contribute to improved sales performance.

In [ ]:
plt.figure(figsize=(8,5))
plt.bar(df['Designation'],df['Total_Sales_Acquired'])
plt.title("Total Sales Acquired and Designation")
plt.xlabel("Designation")
plt.ylabel("Total Sales")
plt.show()

## Part II: Modeling

## 1. Problem formulation

### 1.1 Regression Problem
Salary plays a significant factor in a company's expenditure and employees are always concerned regarding how much salary they should receive. Hence, we want to have the ability to the predict and determine the salary of employees based on various attributes

### 1.2 Classifiction Problem
Employee designation level plays an important role in determining employee responsibilities, salary, and promotions, but companies may lack efficient ways to predict and determine employee's designation level based on various attributes.

### 1.3 Chosen Problem
We will choose to tackle the classification problem of determining employees' designation level. The goal of this project is to create a classification model using the demographic information provided by the dataset can forecast an employee's designation level.

### 1.4 Dependent Variable
Designation Level. It helps determine whether workers are in the correct existing designation levels based on their various attributes

### 2. Model training [30% of Part II]

**a.** Perform feature selection. For each variable, decide if you want to include it as a feature and provide a justification. You may leverage on your analysis in Part I: EDA and/or perform additional analysis.

**Response.** 

**b.** Split the dataset into train and test sets. Describe how you split step by step.

**Response.** 

**c.** State the model(s) you will train, and explain your choice(s), in **no more than 50 words per model**. You only need to
train one model, but if you do train more models, limit yourself to no more than three---Grading is based on the validity and soundness of your model, rather than the quantity.

**Response.** 

**d.** For each model, perform the training, and report the trained parameters and the training scores, if applicable. 

**Response.** 

## 2. Feature Selection

We realised that some of our variables were considered numerical instead of categorical (join designation for example) due to them being a value (1, 2 etc). Thus we changed the values to strings to convert our data to categorical.

In [ ]:
# converting to categorical features
df["Join_Designation"] = df["Join_Designation"].astype(str)
df["Designation"] = df["Designation"].astype(str)
df["Quarterly_Rating"] = df["Quarterly_Rating"].astype(str)
df.dtypes

Next we will split our features to numerical and categorical features

In [ ]:
# splitting the into numerical and categorical
datatypes = df.dtypes
categorical_features = datatypes[datatypes=="object"].index
numerical_features = datatypes[datatypes!="object"].index

### 2.1 Categorical Variables

First, we investigate the distribution of each categorical variable by plotting barplots as below

In [ ]:
n = len(categorical_features)

r, c = 3, 3
fig, ax = plt.subplots(r, c, figsize=(40, 40))

count = 1
for i in range(n):
    if (categorical_features[i] == "Designation"):
        continue
    feature = categorical_features[i]
    ax = plt.subplot(r,c,count)
    count += 1
    ax.set_title(feature)
    df[feature].value_counts(normalize=True).plot(kind='bar')

plt.show()

Based on the boxplots, we can see that the Join_Date and Last_Work_Date on their own do not bring much value to our analysis. We address this by finding the Number_Of_Working_Days by subtracting Join_Date from Last_Work_Date. 

In [ ]:
df['Join_Date'] = pd.to_datetime(df['Join_Date'], format='%d/%m/%Y', errors='coerce')
df['Last_Work_Date'] = pd.to_datetime(df['Last_Work_Date'], format='%d/%m/%Y', errors='coerce')
df["Number_Of_Working_Days"] = (df["Last_Work_Date"] - df["Join_Date"]).dt.days

# Convert the estimated join date back to the original format
df['Join_Date'] = df['Join_Date'].dt.strftime('%d/%m/%Y')
df['Last_Work_Date'] = df['Last_Work_Date'].dt.strftime('%d/%m/%Y')

# adding the number of working days into the numerical list
datatypes = df.dtypes
numerical_features = datatypes[datatypes!="object"].index

fig, ax = plt.subplots(1, 1, figsize=(10, 5))  # Correct way to create figure and axis
ax.set_title("Number of Working Days")
df["Number_Of_Working_Days"].value_counts(normalize=True).plot(kind='bar', ax=ax)
plt.show()

Furthermore, many of our variables have a dominant value. We decided not to select features where its mode has a frequency of more than 50%, as they are likely to have little explanatory power on the variation of `Salary`.

We removed the Date feature as we found it to be irrelevant to the Salary to be determined. (Since the date is simply the date the data was measured, and has no effect on the variables themselves)

In [ ]:
# get 'important' categorical features i.e. those with mode having freq < 50%

categorical_important = []

for feature in categorical_features:
    if (feature == "Date" or feature == "Join_Date" or feature == "Last_Work_Date" or feature == "Designation"):
        continue 
    highest = df[feature].value_counts(normalize=True).iloc[0]
    if highest<0.5:
        categorical_important.append(feature)
print(f"Variables with mode contributing <50% are : {categorical_important}")

Based on the criteria we would keep these three categorical variables.  We next look into how `Deisgnation` is related to these variables.  For each categorical variable, we plot a stacked bar plot and see the distribution of `Designation` with respect to the categorical variable

In [ ]:
n = len(categorical_important)
r, c = 2,2
fig, ax = plt.subplots(r, c, figsize=(20,15))

for i in range(n):
    feature = categorical_important[::-1][i]
    ax = plt.subplot(r,c,i+1)
    pd.crosstab(df[feature], df["Designation"], normalize="index").plot.bar(stacked=True, ax=ax)


Based on a visual observation we noted the following:
- For `Join_Designation`, those who have higher `Join_Designation` have a higher proportion of them having higher `Designation` than those with lower `Join_Designation`

For the other variables, a visual inspection does not reveal any potential relationship to `Designation`

Hence based on an EDA on the categorical variables, we will only keep `Join_Designation`. Furthermore, we would encode the categorical values with a numerical value. We could do that because `Join_Designation` is ordinal, meaning there is a natural order attached to the values.

In [ ]:
# encoding of join designation
df["Join_Designation"] = df["Join_Designation"].astype(float)

categorical_features_selected = ["Join_Designation"]

### 2.2 Numerical Variables

In [ ]:
df[numerical_features].hist(layout=(9,5), figsize=(20,30))

plt.show()

As per visual observation,
- Emp_ID is an index hence it would not exploratory power
- Age has a good variation.
- Salary has a good variation.
- Total_Sales_Acquired and Number_Of_Working_Days have slight variations, which implies their inability to appropriately predict Designation.

To further determine if this would be correct, we must conduct another visual observation by comparing the IQR with the median values.

In [ ]:
features_selected = ["Age", "Total_Sales_Acquired", "Salary", "Number_Of_Working_Days"]

n = len(features_selected)

r, c = 1,4
fig, ax = plt.subplots(r, c, figsize=(20,4))

for i in range(n):
    feature = features_selected[i]
    ax = plt.subplot(r,c,i+1)
    ax.set_title(feature)
    df[['Designation',feature]].boxplot(by='Designation', ax=ax)

plt.show()

We have chosen Age and Salary, given that the IQR adjusts appropriately. The median values increases with the IQR, which implies that the features are correlated to Designation.

Dependent Variable: Designation.

Independent Variable/s: Age and Salary.

Instinctively, we can conclude that the selected features are not correlated with one another. However, to further prove this, we have attached a heatmap that would indicate such.

In [ ]:
features_selected = ["Age", "Salary"]
print(f"Updated Selected Features: {features_selected}")

In [ ]:
r2 = df[features_selected].corr()**2
sns.heatmap(r2)
plt.show()

In [ ]:
# get the feature that has the most number of features
# it is correlated with, beyond certain threshold

def most_corr_feature(data, threshold):
    r2_matrix = abs(data.corr())
    count = r2_matrix[r2_matrix>threshold].count()
    return count.sort_values(ascending=False).index[0]

# return true if all the features are uncorrelated,
# as defined by a threshold

def all_features_uncorr(data, threshold):
    r2_matrix = abs(data.corr())
    n = len(r2_matrix)
    return r2_matrix[r2_matrix>threshold].count().sum()==n

# get a set of uncorrelated features

def get_uncorr_features(data, threshold):
    features = data.columns.tolist()
    while all_features_uncorr(data[features], threshold) == False:
        most_corr_fea = most_corr_feature(data[features], threshold)
        features.remove(most_corr_fea)
    return features

In [ ]:
numerical_features_selected = get_uncorr_features(df[features_selected], 0.5)
print(f"Final Selected Numerical Features : {numerical_features_selected}")

In [ ]:
n = len(numerical_features_selected)

r, c = 1,2
fig, ax = plt.subplots(r, c, figsize=(10,5))

for i in range(n):
    feature = features_selected[i]
    ax = plt.subplot(r,c,i+1)
    ax.set_title(feature)
    df[['Designation',feature]].boxplot(by='Designation', ax=ax)

plt.show()

In [ ]:
features_selected = categorical_features_selected + numerical_features_selected
print(f"Selected features : {features_selected}")

In [ ]:
features_inc_dependent = features_selected + ['Designation']
df[features_inc_dependent].corr()

## 3. Model Evaluation and Selection

**a.** For each model, predict the response variable on the test set.

**Response.** 

**b.** Describe the metric you use to evaluate your model(s). Report the test scores for each model.

The data is separated into training and testing sets using the *train_test_split* function. The function divides it into two sections at random: one for testing (X_test, y_test) and one for training (X_train, y_train), where y is the data we're attempting to predict and X is used to predict target variables. After that, we divide the data into two parts: 75% will be used for training and 25% will be used for testing. 


**c.** If you trained more than one model, identify the final model you would choose for the prediction task, and explain your choice, **in no more than 50 words**.

We ended up using 3 models which are Gaussian Naive Bayes, Logistic Regression and k-Nearest Neighbor.

**Gaussian Naive Bayes**

Since this model assumes feature independence and performs well with tiny datasets, we decided to employ it. It provides a solid foundation for classification activities and is quick and easy to understand.

**Logistic Regression**

Since this model is a popular linear model for classification, we decided to utilize it. It efficiently manages numerical and categorical features and offers probability. 

**k-Nearest Neighbor**

We selected this model because to its ability to efficiently handle mixtures of continuous and categorical numerical data. Additionally, KNN classifies based on similarity, where our feature variables may have a close relationship with our predicted designation values.


In [ ]:
### train_test_split

from sklearn.model_selection import train_test_split

X = df[features_selected]
y = df[["Designation"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
# encoding and scaling
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Convert y to 1D
y_train = y_train.values.ravel()

# Encoding categorical features - one hot encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features_selected)
    ], 
    remainder='passthrough'
)

# Apply transformation before scaling
X_train_encoded = preprocessor.fit_transform(X_train)
X_test_encoded = preprocessor.transform(X_test)

# Apply scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)

# Initialize KNN classifier
knn = KNeighborsClassifier()

# Use GridSearchCV to find the optimal k using cross-validation
grid_search = GridSearchCV(knn, {'n_neighbors': range(1, 16)}, cv=5)  # try k = 1 to k = 15
grid_search.fit(X_train_scaled, y_train)

best_k = grid_search.best_params_['n_neighbors']
print(best_k)


In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

model1 = GaussianNB().fit(X_train, y_train)

model2 = LogisticRegression().fit(X_train, y_train)

model3 = grid_search.best_estimator_
model3.fit(X_train_scaled, y_train)

In [ ]:
## model 1 evaluation

from sklearn.metrics import f1_score, accuracy_score

# predict based on test set

y_pred = model1.predict(X_test)

# compare with ground truth

acc =  accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print("Model 1 Evaluation")
print("-----------------")
print(f"Accuracy: {acc:.3f}")
print(f"F1-score: {f1:.3f}")

For Gaussian Naive Bayes, our trained parameters are the Class Prior Probabilities, Means of features for each class and Variances of Features for each class. For this model, we also have the following training scores: 
- Accuracy -> 0.685
- F1-score -> 0.681


In [ ]:
## model 2 evaluation

# predict based on test set

y_pred = model2.predict(X_test)

# compare with ground truth

acc =  accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print("Model 2 Evaluation")
print("-----------------")
print(f"Accuracy: {acc:.3f}")
print(f"F1-score: {f1:.3f}")

For Logistic Regression, our trained parameters are the Coefficients and Intercept. For this model, we also have the following training scores:
- Accuracy -> 0.503
- F1-score -> 0.499

In [ ]:
## model 3 evaluation

# predict based on test set
y_pred = model3.predict(X_test_scaled)

# compare with ground truth

acc =  accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print("Model 3 Evaluation")
print("-----------------")
print(f"k-value: {best_k}")
print(f"Accuracy: {acc:.3f}")
print(f"F1-score: {f1:.3f}")

For k-Nearest Neighbor, our trained parameters is the k-value, where we found the best k-value using grid search. 
- Optimal k -> 14
  
For this model, we also have the following training scores:
- Accuracy -> 0.850
- F1-score -> 0.847

## 4. Findings and conclusion

### 4.1 Interpretation of Model and Insights
Our final model implies that an increase in `Join_Designation`, `Salary` or `Age` would result in a higher `Designation`. This is consistent with intuition, as a higher `Age` implies more experience, and higher `Salary` and `Join_Designation` imply they are good in their work. This would logically result in a higher `Designation`.

### 4.2 Lessons Learnt From Project
We learned how to filter the variables that are important to our analysis, as not every variable is important. We also learned about the importance of statistical analysis in the real world, in the case of our project, to visualise and predict a very real variable people would want to predict.

## 5. Non-technical protocol

**a.** Describe the detailed contribution of each team member, including both the tangible (e.g., implementation, testing, writing) and intangible (e.g., generating ideas, planning, leadership) efforts.

**Response.** 

**b.** List any references and sources you have cited.

GridSearchCV documentation - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
KNeighborClassifier documentation - https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier